In [1]:
# Bibliotecas Importadas
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pnd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import ExtraTreeClassifier

In [ ]:
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]
#Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)
#Chama o dataframe contido na variável chamada df
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
#Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
#Filtra meterSN diferente de '>N<A'
df = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
#Garante que todas as linhas com gain nulo sejam preenchidas com 1. Não é garantido que é o valor correto, mas é o melhor que podemos fazer
df['gain'].fillna(1, inplace=True)
#Corrige os pulsos para m²
df['pulseCount'] = df['pulseCount'] * df['gain']
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
df['datetime'] = pnd.to_datetime(df['datetime'])
df['dateTimeSegundos'] = df['datetime'].astype(np.int64) // 10**9
df['diffDateTime'] = df.groupby(['clientCode', 'meterSN']).dateTimeSegundos.diff()
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
df['diffPulseCountTempo'] = df['diffPulseCount'] / df['diffDateTime']
#Preenche os valores nulos (iniciais) com 0
df['diffDateTime'].fillna(0, inplace=True) 
df['diffPulseCount'].fillna(0, inplace=True)
df['diffPulseCountTempo'].fillna(0, inplace=True)
#Reseta o index
df.reset_index(drop=True, inplace=True)
#Seleciona as colunas que serão usadas
df = df[['clientCode', 'meterSN', "pulseCount", 'diffPulseCount','datetime', 'diffDateTime', 'diffPulseCountTempo', 'dateTimeSegundos']]
#Calcula a média e o desvio padrão do diffPulseCount por cliente
df['mediaCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('mean')
df['desvioPadraoCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('std')
df['diffDateTime'].describe()

In [ ]:
dfCliente = df[df['clientCode'].str.contains('b484c8b5ec41d7d82abdfda82fd5f14c076db223dbd2bc')]
dfCliente.sort_values(by='datetime', inplace=True)
dfCliente.reset_index(drop=True, inplace=True)
dfCliente

# Análise de anomalias


### Início com todos os casos classificados como normal

In [ ]:
df.sort_values(by='diffDateTime', inplace=True, ascending=False)
df.reset_index(drop=True, inplace=True)
df[df['diffDateTime'] < 90000]

In [ ]:
# Filtrando e ordenando os dados por cliente e tempo
dfCliente = df[df['clientCode'].str.contains('9479bbab64308ce0789119e6dc874805d07f386dd45c98')]
dfCliente.sort_values(by='datetime', inplace=True)

# Plotando o gráfico de diffDateTime ao longo do tempo
plt.figure(figsize=(12, 6))
plt.plot(dfCliente['datetime'], dfCliente['diffDateTime'], marker='o', linestyle='-', color='r')
plt.title('Variação do diffDateTime ao longo do tempo')
plt.xlabel('Data e Hora')
plt.ylabel('Diff DateTime (segundos)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

In [ ]:
# Filtrando e ordenando os dados por cliente e tempo
dfCliente = df[df['clientCode'].str.contains('b4e5384402e135d94a96529f98c6eda68017dd25628243')]
dfCliente.sort_values(by='datetime', inplace=True)

# Plotando o gráfico de diffDateTime ao longo do tempo
plt.figure(figsize=(12, 6))
plt.plot(dfCliente['datetime'], dfCliente['diffPulseCountTempo'], marker='o', linestyle='-', color='r')
plt.title('Variação do diffDateTime ao longo do tempo')
plt.xlabel('Data e Hora')
plt.ylabel('Diff DateTime (segundos)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

In [ ]:
dfCliente.sort_values(by='diffDateTime', inplace=True, ascending=False)
dfCliente.reset_index(drop=True, inplace=True)
dfCliente[dfCliente['diffDateTime'] < 90000]

In [ ]:
# Filtrando e ordenando os dados por cliente e tempo
dfCliente = df[df['clientCode'].str.contains('41462ff695d437dfe98a3379e01b48773da62c8f418889')]
dfCliente.sort_values(by='datetime', inplace=True)

# Plotando o gráfico de diffDateTime ao longo do tempo
plt.figure(figsize=(12, 6))
plt.plot(dfCliente['datetime'], dfCliente['diffDateTime'], marker='o', linestyle='-', color='r')
plt.title('Variação do diffDateTime ao longo do tempo')
plt.xlabel('Data e Hora')
plt.ylabel('Diff DateTime (segundos)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

In [ ]:
df['mediaPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('mean')
df['desvioPadraoPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('std')

df['tipo'] = "c"

#Classifica os inidivíduos sem medições por longos períodos de tempo
df.loc[df['diffDateTime'] > 86400, 'tipo'] = "sm1"
df.loc[df['diffDateTime'] > 604800, 'tipo'] = "sm7"
df.loc[df['diffDateTime'] > 2592000, 'tipo'] = "sm30"

#Classifica consumo acima de 3 desvio padrão, consumo negativo e consumo zerado
df.loc[df['diffPulseCountTempo'] > df['mediaPCTCliente'] + 3 * df['desvioPadraoPCTCliente'], 'tipo'] = "dp3"
df.loc[df['diffPulseCountTempo'] < 0, 'tipo'] = "cn"
df.loc[(df['pulseCount'] == 0) & (df['diffPulseCountTempo'] < 0), 'tipo'] = "cz"

print(df.tipo.value_counts())
df

In [10]:
df.dropna(inplace=True)

### Modelo DecisionTreeClassifier

In [ ]:
from sklearn.utils import resample
# Selecionando as variáveis de entrada (features) e a variável alvo (target)
X = df[['diffPulseCountTempo', 'mediaPCTCliente', 'desvioPadraoPCTCliente', 'diffDateTime']]  # Features
y = df['tipo']  # Target

# Dividindo os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_majority = X_test[y_test == 'c']
X_minority = X_test[y_test != 'c']
y_majority = y_test[y_test == 'c']
y_minority = y_test[y_test != 'c']
X_majority_downsampled, y_majority_downsampled = resample(X_majority, y_majority, replace=False, n_samples=len(X_minority), random_state=42)
X_test_balanced = pnd.concat([X_majority_downsampled, X_minority])
y_test_balanced = pnd.concat([y_majority_downsampled, y_minority])

# Criando e treinando o modelo de Árvore de Decisão
clf = DecisionTreeClassifier(random_state=42, class_weight='balanced')
clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = clf.predict(X_test_balanced)

# Avaliando o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test_balanced, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test_balanced, y_pred))


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
grid_param ={
    'criterion': ['gini', 'entropy'],
    'max_depth' : [None, 10,30,50,100],
    'min_samples_leaf' : [1, 2, 10],
    'min_samples_split': [2, 5, 10],
    'splitter' : ['best', 'random'],
    'max_features': [None, 'sqrt', 'log2'],
    'random_state': [42]
}
modeloAParametrar = DecisionTreeClassifier()
modeloParametrado = GridSearchCV(estimator=modeloAParametrar, param_grid=grid_param, cv=5, n_jobs=-1, scoring=['accuracy', 'precision'], refit='precision', verbose=2)
modeloParametrado.fit(X_train, y_train)
print("Melhores parâmetros:\n", modeloParametrado.best_params_)
y_pred = modeloParametrado.predict(X_test)
print('****************************************')
print(classification_report(y_test, y_pred))
print('****************************************')

### Modelo Random Forest 

In [ ]:
# Selecionando as variáveis de entrada (features) e a variável alvo (target)
X = df[['diffPulseCountTempo', 'mediaPCTCliente', 'desvioPadraoPCTCliente', 'diffDateTime']]  # Features
y = df['tipo']  # Target

# Dividindo os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando e treinando o modelo de Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs = -1)
rf_clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_rf = rf_clf.predict(X_test)

# Avaliando o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_rf))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_rf))


### Modelo Extra Trees Classifier

In [ ]:
#verificando e excluindo valores nulos do desvioPadraoPCTCliente
print (df['desvioPadraoPCTCliente'].isna().sum())
df = df.dropna(subset=['desvioPadraoPCTCliente'])
print (df['desvioPadraoPCTCliente'].isna().sum())

In [ ]:
# Importando as bibliotecas necessárias
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Selecionando as variáveis de entrada (features) e a variável alvo (target)
X = df[['diffPulseCountTempo', 'mediaPCTCliente', 'desvioPadraoPCTCliente', 'diffDateTime']]  # Features
y = df['tipo']  # Target

# Dividindo os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando e treinando o modelo de Extra Trees
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1)
et_clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_et = et_clf.predict(X_test)

# Avaliando o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_et))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_et))


### Modelo Extra Tree Classifier

In [ ]:
# Selecionando as variáveis de entrada (features) e a variável alvo (target)
X = df[['diffPulseCountTempo', 'mediaPCTCliente', 'desvioPadraoPCTCliente', 'diffDateTime']]  # Features
y = df['tipo']  # Target

# Dividindo os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando e treinando o modelo Extra Tree
et_clf = ExtraTreeClassifier(random_state=42)
et_clf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_et = et_clf.predict(X_test)

# Avaliando o modelo
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred_et))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred_et))